In [1]:
! pip install pillow opencv-python
! pip freeze > requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 23.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 33.4 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
from pathlib import Path
from typing import List, Tuple

from PIL import Image

from utils.processor import ImageAnnotatorController

In [3]:
raw_data_path = Path("data/raw")

In [4]:
def crop_images(image_paths: List[Path], 
                output_dir: Path, 
                crop_size: Tuple[int, int] = (640, 640), 
                overlap: int = 0) -> None:
    """Crops all images in the provided list of paths into smaller 640x640 tiles and saves them.
    
    Args:
        image_paths (List[Path]): List of paths to the images.
        output_dir (Path): Directory where the cropped images will be saved.
        crop_size (Tuple[int, int], optional): The target size for cropping (width, height). 
                                               Defaults to (640, 640).
        overlap (int, optional): Overlap size between crops to achieve additional tiles. Defaults to 320.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    crop_width, crop_height = crop_size
    for img_path in image_paths:
        with Image.open(img_path) as img:
            img_width, img_height = img.size
            crop_count = 0
            step_x = crop_width - overlap
            step_y = crop_height - overlap
            for top in range(0, img_height - crop_height + 1, step_y):
                for left in range(0, img_width - crop_width + 1, step_x):
                    right = left + crop_width
                    bottom = top + crop_height
                    cropped_img = img.crop((left, top, right, bottom))
                    crop_count += 1
                    save_path = output_dir / f"{img_path.stem}_crop_{crop_count}.jpg"
                    cropped_img.save(save_path)
                    print(f"Cropped and saved image: {save_path}")
            
            print(f"Total crops for {img_path.name}: {crop_count}")

In [5]:
# image_paths = list(raw_data_path.glob('*.JPG'))
# output_directory = Path("data/cropped_images")
# crop_images(image_paths, output_directory)

In [7]:
IMAGE_PATH = 'data/cropped_images/73_crop_88.jpg'
OUTPUT_DIR = 'data/cropped_labels'
os.makedirs(OUTPUT_DIR, exist_ok=True)

annotator = ImageAnnotatorController(IMAGE_PATH, OUTPUT_DIR)
annotator.run()

Only C and default locale supported with the posix collation implementation
Only C and default locale supported with the posix collation implementation
Case insensitive sorting unsupported in the posix collation implementation
Numeric mode unsupported in the posix collation implementation


: 